In [51]:
import json, requests
import Levenshtein as lev
import unicodedata
import pandas as pd
import urllib
import os

In [52]:

ID = "aaf20ac1"
key = "7bd88c15c64569da8de3c414ace8d236"

baseURL = "http://api.yummly.com/v1/api/recipes?_app_id="+ID+"&_app_key="+key+"&q=@&maxResult=3"
baseURLGet = "http://api.yummly.com/v1/api/recipe/@?_app_id="+ID+"&_app_key="+key

noImages = []

In [53]:
def convertName(name):
    converted = unicodedata.normalize('NFKD', name).encode('ascii','ignore')
    return converted.lower()

def bestMatch(query, response):
    tmpQuery = query.lower()
    levDistances = [lev.distance(convertName(match["recipeName"]), tmpQuery) for match in response["matches"]]
    return response["matches"][levDistances.index(min(levDistances))]

def getRecipe(candidateID):
    url = baseURLGet.replace("@", candidateID)
    response = json.loads(requests.get(url).text)
    return response

def buildJson(original, searchResponse, recipeResponse):
    jsonBuilder = {}

    jsonBuilder["RecipeURL"] = recipeResponse["source"]["sourceRecipeUrl"]
    jsonBuilder["CleanIngredients"] = searchResponse["ingredients"]
    jsonBuilder["RawIngredients"] = recipeResponse["ingredientLines"]
    jsonBuilder["RecipeName"] = searchResponse["recipeName"]
    jsonBuilder["FoodName"] = original
    return jsonBuilder

def rangeMatch(query, response):
    tmpQuery = query.lower()
    levDistances = [lev.distance(convertName(match["recipeName"]), tmpQuery) for match in response["matches"]]
    response["matches"].pop(levDistances.index(min(levDistances)))
    return response["matches"]

def saveImageFromUrl(query, recipeName, index, lstURL):
    tmpQuery = (query.replace(" ", "_")).lower()
    tmpRecipeName = (recipeName.replace(" ", "_")).lower()
    
    for candidate in ["hostedLargeUrl", "hostedMediumUrl", "hostedSmallUrl"]:
        try:
            directory = "food101Pictures/"+tmpQuery
            if not os.path.exists(directory):
                os.makedirs(directory)
                
            fileName = directory+"/"+str(index)+"_"+tmpRecipeName+"_"+candidate+".jpg"
            urllib.urlretrieve(lstURL[0][candidate], fileName)
            break
        except:
            pass

    
def buildWideJson(original, searchResponse, recipeResponse, index):
    jsonBuilder = {}
    jsonBuilder["RecipeName"] = searchResponse["recipeName"]
    jsonBuilder["RecipeURL"] = recipeResponse["source"]["sourceRecipeUrl"]
    jsonBuilder["CleanIngredients"] = searchResponse["ingredients"]
    jsonBuilder["RawIngredients"] = recipeResponse["ingredientLines"]

    if recipeResponse["images"]:
        saveImageFromUrl(original, searchResponse["recipeName"], index, recipeResponse["images"])
        jsonBuilder["Images"] = recipeResponse["images"]
    else:
        noImages.add(original+" "+searchResponse["recipeName"])
        
    return jsonBuilder
            

In [6]:
def searchFromList():
    responses = []
    notFound = []
    foodList = pd.read_csv("food101.csv", sep=',')
    for query in foodList['FoodName']:
        print "query: ", query
        params = query.lower().replace(" ", "+")
        url = baseURL.replace("@",params)
        response = json.loads(requests.get(url).text)
        if not response["matches"]:
            "Hello no matches :3"
            notFound.append(query)
    
        else:
            candidate = bestMatch(query, response)
            candidateID = candidate["id"]
            recipeResponse = getRecipe(candidateID)
            builtJson = buildJson(query, candidate, recipeResponse)
            responses.append(builtJson)
            
    data = json.dumps(responses)
    with open('recipesData.json', 'w') as f:
        json.dump(responses, f)
    with open('notFound.json', 'w') as f:
        json.dump(notFound, f)
    
    return responses, notFound
    
responses, notFound = searchFromList()

#print responses
#print notFound

query:  Apple pie
query:  Baby back ribs
query:  Baklava
query:  Beef carpaccio
query:  Beef tartare
query:  Beet salad
query:  Beignets
query:  Bibimbap
query:  Bread pudding
query:  Breakfast burrito
query:  Bruschetta
query:  Caesar salad
query:  Cannoli
query:  Caprese salad
query:  Carrot cake
query:  Ceviche
query:  Cheesecake
query:  Cheese plate
query:  Chicken curry
query:  Chicken quesadilla
query:  Chicken wings
query:  Chocolate cake
query:  Chocolate mousse
query:  Churros
query:  Clam chowder
query:  Club sandwich
query:  Crab cakes
query:  Creme brulee
query:  Croque madame
query:  Cupcakes
query:  Deviled eggs
query:  Donuts
query:  Dumplings
query:  Edamame
query:  Eggs benedict
query:  Escargots
query:  Falafel
query:  Filet mignon
query:  Fish and chips
query:  Foie gras
query:  French fries
query:  French onion soup
query:  French toast
query:  Fried calamari
query:  Fried rice
query:  Frozen yogurt
query:  Garlic bread
query:  Gnocchi
query:  Greek salad
query:  Gr

In [50]:
def searchRecipesRange():
    notFound = []
    foodList = pd.read_csv("food101.csv", sep=',')
    jsonBuilder = {}
    for query in foodList['FoodName']:
        print "query: ", query
        params = query.lower().replace(" ", "+")
        url = baseURL.replace("@",params)
        response = json.loads(requests.get(url).text)
        if not response["matches"]:
            "Hello no matches :3"
            notFound.append(query)
    
        else:
            matches = rangeMatch(query, response)
            lst = []
            i = 0
            for match in matches:
                candidateID = match["id"]
                recipeResponse = getRecipe(candidateID)
                builtJson = buildWideJson(query, match, recipeResponse, i)
                lst.append(builtJson)
                i+=1
                
            jsonBuilder[query] = lst
            
    data = json.dumps(jsonBuilder)
    with open('recipesWideData.json', 'w') as f:
        json.dump(jsonBuilder, f)
    with open('notFoundWide.json', 'w') as f:
        json.dump(notFound, f)
    
    return jsonBuilder, notFound
    
jsonBuilt, notFound = searchRecipesRange()

query:  pizza margarita
query:  tomato soup
query:  onion soup
query:  chicken with spinachs
query:  spaghetti bolognese
query:  NOTFOUNDPLS
query:  ASDASDjashdjshdj
